# Imports data from tissue analyser and performs some further analysis

In [1]:
!pip install numpy
!pip install pandas
!pip install xarray
!pip install scipy
!pip install matplotlib
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# load external packages

from curses import meta
import os
import os.path
import argparse

import logging
import warnings
import time
from copy import deepcopy
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr

import math
from scipy.optimize import curve_fit
from scipy.interpolate import griddata

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import seaborn as sns

import yaml
from PIL import Image
from PIL.TiffTags import TAGS

In [3]:
# load segmentation scripts

from import_data_function import import_data
from utils import *

In [4]:
# initialize logger

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s :: %(levelname)s :: %(message)s')
log = logging.getLogger("DataImport")
log.setLevel(logging.DEBUG)

log.info("Initialized logger")

2023-03-30 12:03:57,026 :: INFO :: Initialized logger


In [5]:
cells, bonds = import_data(config_file="sample-data/import_config.yml", log=log)

2023-03-30 12:03:57,104 :: INFO :: Loading data at /home/julian/tissue_segmentation/bp-segmentation
2023-03-30 12:03:57,162 :: INFO :: Using channel sum_px_intensity_cells_ch1 for HCA criterion
2023-03-30 12:03:57,166 :: INFO :: Detected 22.37 perc. HC
2023-03-30 12:03:57,171 :: INFO :: HC density per file: 
filename
E14_75I_6_0.tif    0.223684
dtype: float64
2023-03-30 12:03:57,208 :: INFO :: Setting column aligned_with_PD_axis: `True' (from default).
2023-03-30 12:03:57,210 :: INFO :: Loaded data at /home/julian/tissue_segmentation/bp-segmentation in 0.10s.
2023-03-30 12:03:57,211 :: INFO :: ---------
2023-03-30 12:03:57,212 :: INFO :: Important Info:
2023-03-30 12:03:57,212 :: INFO :: 
2023-03-30 12:03:57,215 :: INFO :: HCs have an average HCA criterion of: 
file_id
0    255.0
Name: HCA_criterion, dtype: float64
2023-03-30 12:03:57,227 :: INFO :: Gathered 608 cells (incl. 136 HC) at stages ['E14'] and positions ['75I'] from 1 files: 
 ['E14_75I_6_0.tif']
2023-03-30 12:03:57,228 :: I

In [10]:
print(cells.to_string())

     index         filename  frame_nb  local_id_cells  area_cells  sum_px_intensity_cells_ch1  sum_px_intensity_cells_ch2  sum_px_intensity_cells_ch3  sum_px_intensity_cells_12_bits  first_pixel_x_cells  first_pixel_y_cells  center_x_cells  center_y_cells  s1_stretch_cells  s2_stretch_cells  s0_stretch_cells  orientation_cells                                                                  vx_coords_cells  nb_of_perimeter_pixels  perimeter_length  sum_px_intensity_perimeter_ch1  sum_px_intensity_perimeter_ch2  sum_px_intensity_perimeter_ch3  sum_px_intensity_12_perimeter_bits  p1_polarity_ch1  p2_polarity_ch1  p1_polarity_ch2  p2_polarity_ch2  p1_polarity_ch3  p2_polarity_ch3  p1_polarity_12bits  p2_polarity_12bits  p1_polarity_ch1_normalized  p2_polarity_ch1_normalized  p1_polarity_ch2_normalized  p2_polarity_ch2_normalized  p1_polarity_ch3_normalized  p2_polarity_ch3_normalized  p1_polarity_12bits_normalized  p2_polarity_12bits_normalized  deprecated_v1_polarity_ch1  deprecated_v2_p

In [11]:
print(bonds.to_string())

      index         filename  frame_nb  local_id_bonds  bond_size_in_px  bond_length_in_px  bond_orientation  vx_1_x  vx_1_y  vx_2_x  vx_2_y  sum_px_int_vertices_excluded_ch1  sum_px_int_vertices_excluded_ch2  sum_px_int_vertices_excluded_ch3  sum_px_int_vertices_excluded_12bits  sum_px_int_vertices_included_ch1  sum_px_int_vertices_included_ch2  sum_px_int_vertices_included_ch3  sum_px_int_vertices_included_12bits  first_pixel_x_bonds  first_pixel_y_bonds  cell_id_around_bond1  cell_id_around_bond2  Unnamed: 22                                          filepath stage position  sample_id crop_id  file_id  global_id_bonds  global_cell_id_around_bond1  global_cell_id_around_bond2  aligned_with_PD_axis  PD_position SI_position  resolution_x  resolution_y  resolution_scale             type  normalized_bond_length  ppMLC_foci_X  ppMLC_foci_Y  ppMLC_foci_distance  ppMLC_foci_Size  ppMLC_foci_Max  ppMLC_foci_Total
0         0  E14_75I_6_0.tif         9               1                2         

In [15]:
print("Available datasets in cells: ")
for set in cells.columns:
  print("  {}".format(set))

Available datasets in cells: 
  index
  filename
  frame_nb
  local_id_cells
  area_cells
  sum_px_intensity_cells_ch1
  sum_px_intensity_cells_ch2
  sum_px_intensity_cells_ch3
  sum_px_intensity_cells_12_bits
  first_pixel_x_cells
  first_pixel_y_cells
  center_x_cells
  center_y_cells
  s1_stretch_cells
  s2_stretch_cells
  s0_stretch_cells
  orientation_cells
  vx_coords_cells
  nb_of_perimeter_pixels
  perimeter_length
  sum_px_intensity_perimeter_ch1
  sum_px_intensity_perimeter_ch2
  sum_px_intensity_perimeter_ch3
  sum_px_intensity_12_perimeter_bits
  p1_polarity_ch1
  p2_polarity_ch1
  p1_polarity_ch2
  p2_polarity_ch2
  p1_polarity_ch3
  p2_polarity_ch3
  p1_polarity_12bits
  p2_polarity_12bits
  p1_polarity_ch1_normalized
  p2_polarity_ch1_normalized
  p1_polarity_ch2_normalized
  p2_polarity_ch2_normalized
  p1_polarity_ch3_normalized
  p2_polarity_ch3_normalized
  p1_polarity_12bits_normalized
  p2_polarity_12bits_normalized
  deprecated_v1_polarity_ch1
  deprecated_v2_pola

In [14]:
print("Available datasets in bonds: ")
for set in bonds.columns:
  print("  {}".format(set))

Available datasets in bonds: 
  index
  filename
  frame_nb
  local_id_bonds
  bond_size_in_px
  bond_length_in_px
  bond_orientation
  vx_1_x
  vx_1_y
  vx_2_x
  vx_2_y
  sum_px_int_vertices_excluded_ch1
  sum_px_int_vertices_excluded_ch2
  sum_px_int_vertices_excluded_ch3
  sum_px_int_vertices_excluded_12bits
  sum_px_int_vertices_included_ch1
  sum_px_int_vertices_included_ch2
  sum_px_int_vertices_included_ch3
  sum_px_int_vertices_included_12bits
  first_pixel_x_bonds
  first_pixel_y_bonds
  cell_id_around_bond1
  cell_id_around_bond2
  Unnamed: 22
  filepath
  stage
  position
  sample_id
  crop_id
  file_id
  global_id_bonds
  global_cell_id_around_bond1
  global_cell_id_around_bond2
  aligned_with_PD_axis
  PD_position
  SI_position
  resolution_x
  resolution_y
  resolution_scale
  type
  normalized_bond_length
  ppMLC_foci_X
  ppMLC_foci_Y
  ppMLC_foci_distance
  ppMLC_foci_Size
  ppMLC_foci_Max
  ppMLC_foci_Total
